In [ ]:
# cleaning...%load_ext autoreload
# cleaning...%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
assert SETTING["signal_type"] == "context"

In [ ]:
if "feature" in SETTING["context_element_types"]:

    feature_context = pd.read_csv(
        PATH["feature_x_sample.feature_context.tsv"], sep="\t", index_col=0
    )

    feature_context.columns.name = SETTING["sample_alias"]

    feature_context_negative = feature_context.clip(upper=0)

    feature_context_positive = feature_context.clip(lower=0)

In [ ]:
if "sample" in SETTING["context_element_types"]:

    sample_context = pd.read_csv(
        PATH["feature_x_sample.sample_context.tsv"], sep="\t", index_col=0
    ).T

    sample_context.index.name = SETTING["sample_alias"]

    sample_context_negative = sample_context.clip(upper=0)

    sample_context_positive = sample_context.clip(lower=0)

In [ ]:
if (
    "feature" in SETTING["context_element_types"]
    and "sample" in SETTING["context_element_types"]
):

    context_negative = kraft.normalize_dataframe(
        feature_context_negative, 1, "0-1"
    ) + kraft.normalize_dataframe(sample_context_negative, 0, "0-1")

    context_positive = kraft.normalize_dataframe(
        feature_context_positive, 1, "0-1"
    ) + kraft.normalize_dataframe(sample_context_positive, 0, "0-1")

elif "feature" in SETTING["context_element_types"]:

    context_negative = feature_context_negative

    context_positive = feature_context_positive

elif "sample" in SETTING["context_element_types"]:

    context_negative = sample_context_negative

    context_positive = sample_context_positive

In [ ]:
layout = {"yaxis": {"title": "Signal Sum"}}

In [ ]:
standard_deviation = SETTING["select_feature_standard_deviation"]

if SETTING["select_feature_automatically"]:

    context_negative_features = kraft.select_series_indices(
        context_negative.sum(axis=1),
        "<",
        standard_deviation=standard_deviation,
        layout=layout,
    )

    context_positive_features = kraft.select_series_indices(
        context_positive.sum(axis=1),
        ">",
        standard_deviation=standard_deviation,
        layout=layout,
    )

else:

    context_negative_features = (
        context_negative.replace(0, np.nan).dropna(how="all").index
    )

    context_positive_features = (
        context_positive.replace(0, np.nan).dropna(how="all").index
    )

In [ ]:
standard_deviation = SETTING["select_feature_standard_deviation"]

if SETTING["select_sample_automatically"]:

    context_negative_samples = kraft.select_series_indices(
        context_negative.sum(),
        "<",
        standard_deviation=standard_deviation,
        layout=layout,
    )

    context_positive_samples = kraft.select_series_indices(
        context_positive.sum(),
        ">",
        standard_deviation=standard_deviation,
        layout=layout,
    )

else:

    context_negative_samples = (
        context_negative.replace(0, np.nan).dropna(how="all").columns
    )

    context_positive_samples = (
        context_positive.replace(0, np.nan).dropna(how="all").columns
    )

In [ ]:
context_negative_signal = context_negative.loc[
    context_negative_features, context_negative_samples
]

context_positive_signal = context_positive.loc[
    context_positive_features, context_positive_samples
]

In [ ]:
if "negative" in SETTING["contexts"] and "positive" in SETTING["contexts"]:

    context_negative_signal.index = pd.Index(
        ("(-) {}".format(feature) for feature in context_negative_signal.index),
        name=context_negative_signal.index.name,
    )

    context_positive_signal.index = pd.Index(
        ("(+) {}".format(feature) for feature in context_positive_signal.index),
        name=context_positive_signal.index.name,
    )

    context_signal = pd.concat((-context_negative_signal, context_positive_signal))

elif "negative" in SETTING["contexts"]:

    context_signal = -context_negative_signal

elif "positive" in SETTING["contexts"]:

    context_signal = context_positive_signal

In [ ]:
if SETTING["signal_normalization_method"] is not None:

    context_signal = kraft.normalize_dataframe(
        context_signal,
        SETTING["signal_normalization_axis"],
        SETTING["signal_normalization_method"],
    )

In [ ]:
context_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

kraft.summarize_feature_x_sample(
    context_signal,
    feature_x_sample_alias="Context Signal",
    feature_x_sample_value_name="Context Signal",
)